## Exercise 2

We'll first just copy the code from the last exercise to start where we left off.

In [1]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

# Exercise 1.1
product_data = pd.read_csv('../Data/products.csv')

# Exercise 1.2
product_data['market_size'] = product_data['city_population'] * 90
product_data['market_share'] = product_data['servings_sold'] / product_data['market_size']
product_data['outside_share'] = 1 - product_data.groupby('market')['market_share'].transform('sum')

# Exercise 1.3
product_data['logit_delta'] = np.log(product_data['market_share'] / product_data['outside_share'])
statsmodels_ols = smf.ols('logit_delta ~ 1 + mushy + price_per_serving', product_data)
statsmodels_results = statsmodels_ols.fit(cov_type='HC0')

# Exercise 1.4
product_data = product_data.rename(columns={
    'market': 'market_ids',
    'product': 'product_ids',
    'market_share': 'shares',
    'price_per_serving': 'prices',
})
product_data['demand_instruments0'] = product_data['prices']
ols_problem = pyblp.Problem(pyblp.Formulation('1 + mushy + prices'), product_data)
ols_results = ols_problem.solve(method='1s')

# Exercise 1.5
fe_problem = pyblp.Problem(pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), product_data)
fe_results = fe_problem.solve(method='1s')

# Exercise 1.6
first_stage = smf.ols('prices ~ 0 + price_instrument + C(market_ids) + C(product_ids)', product_data)
first_stage_results = first_stage.fit(cov_type='HC0')
product_data = product_data.drop(columns='demand_instruments0').rename(columns={'price_instrument': 'demand_instruments0'})
iv_problem = pyblp.Problem(pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), product_data)
iv_results = iv_problem.solve(method='1s')

# Exercise 1.7
counterfactual_market = 'C01Q2'
counterfactual_data = product_data.loc[product_data['market_ids'] == counterfactual_market, ['product_ids', 'mushy', 'prices', 'shares']]
counterfactual_data['new_prices'] = counterfactual_data['prices']
counterfactual_data.loc[counterfactual_data['product_ids'] == 'F1B04', 'new_prices'] /= 2
counterfactual_data['new_shares'] = iv_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['iv_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']

# Exercise 1.8
iv_elasticities = iv_results.compute_elasticities(market_id=counterfactual_market)

### 1. Describe cross-market variation

Let's first look at the amount of cross-market choice set variation.

In [2]:
choice_variation = product_data.groupby('market_ids', as_index=False).agg(**{
    'products': ('product_ids', 'count'),
    'mushy_mean': ('mushy', 'mean'),
    'mushy_std': ('mushy', 'std'),
    'prices_mean': ('prices', 'mean'),
    'prices_std': ('prices', 'std'),
})
choice_variation.describe()

,products,mushy_mean,mushy_std,prices_mean,prices_std
count,94.0,9.400e+01,9.400e+01,94.000,94.000
mean,24.0,3.333e-01,4.815e-01,0.126,0.029
std,0.0,5.581e-17,1.116e-16,0.005,0.004
min,24.0,3.333e-01,4.815e-01,0.112,0.022
25%,24.0,3.333e-01,4.815e-01,0.122,0.026
50%,24.0,3.333e-01,4.815e-01,0.126,0.028
75%,24.0,3.333e-01,4.815e-01,0.129,0.031
max,24.0,3.333e-01,4.815e-01,0.138,0.038


The same 20 products are present in each market, and mushyness does not change across markets. So there is no cross-market variation in the number of products or mushyness. However, there is cross-market variation in prices. Recalling the linear regression intuition for identification, this means that we have hope of credibly identifying the amount of unobserved preference heterogeneity for price, but not for a constant or mushy.

Next, let's load the new demographic data.

In [3]:
demographic_data = pd.read_csv('../Data/demographics.csv').rename(columns={'market': 'market_ids'})
demographic_data.sample(n=5, random_state=0)

,market_ids,quarterly_income
511,C18Q2,3785.817
124,C05Q1,4359.937
217,C08Q1,3822.916
1426,C47Q2,1708.163
891,C32Q1,3983.035


Income has a long right tail, so it's easier to work in logs.

In [4]:
demographic_data['log_income'] = np.log(demographic_data['quarterly_income'])
demographic_data[['quarterly_income', 'log_income']].describe()

,quarterly_income,log_income
count,1880.000,1880.000
mean,4575.000,8.091
std,3679.854,0.940
min,23.411,3.153
25%,2128.625,7.663
50%,3836.111,8.252
75%,5884.614,8.680
max,33724.436,10.426


Let's see how much cross-market demographic variation there is.

In [5]:
demographic_variation = demographic_data.groupby('market_ids', as_index=False).agg(**{
    'log_income_mean': ('log_income', 'mean'),
    'log_income_std': ('log_income', 'std'),
})
demographic_variation.describe()

,log_income_mean,log_income_std
count,94.000,94.000
mean,8.091,0.885
std,0.289,0.242
min,7.499,0.523
25%,7.872,0.710
50%,8.093,0.831
75%,8.338,1.086
max,8.622,1.539


There seems to be a good amount of cross-market demographic variation. Consumers' income varies a good amount across our markets. This means that using this variation, we have a hope of credibly estimating how income shifts the preference of different characteristics. However, because we have market fixed effects, a parameter on income alone would be collinear and not identified.

### 2. Estimate a parameter on mushy $\times$ log income

First, we need consumer type data with demographic draws, standard normal nodes (for later), and uniform weights.

In [6]:
agent_data = demographic_data[['market_ids', 'log_income']].groupby('market_ids', as_index=False).sample(n=1000, replace=True, random_state=0)
agent_data[['nodes0', 'nodes1', 'nodes2']] = np.random.default_rng(seed=0).normal(size=(len(agent_data), 3))
agent_data['weights'] = 1 / agent_data.groupby('market_ids').transform('size')
agent_data.sample(n=5, random_state=0)

,market_ids,log_income,nodes0,nodes1,nodes2,weights
1100,C37Q2,7.590,-1.401,1.667,-1.312,0.001
1860,C65Q2,7.584,-1.609,0.747,-0.582,0.001
1726,C58Q1,7.949,-0.914,1.266,0.043,0.001
491,C18Q1,8.239,-0.946,-0.069,-1.075,0.001
314,C12Q2,8.569,-1.268,-0.034,-0.462,0.001


We also need a new instrument.

In [7]:
product_data = product_data.merge(demographic_variation[['market_ids', 'log_income_mean']], on='market_ids')
product_data['demand_instruments1'] = product_data['log_income_mean'] * product_data['mushy']

To model an interaction between mushy and the log of income, we need two new formulations when setting up the problem.

In [8]:
product_formulations = (pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), pyblp.Formulation('0 + mushy'))
agent_formulation = pyblp.Formulation('0 + log_income')
mushy_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
mushy_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED 
---  ----  -----  ----  ----  ---  ----  ----
94   2256  94000   1     1     1    2     2  

Formulations:
       Column Indices:             0     
-----------------------------  ----------
 X1: Linear Characteristics      prices  
X2: Nonlinear Characteristics    mushy   
       d: Demographics         log_income

Now that we have a nonlinear GMM estimator, we need a nonlinear optimizer. We'll use SciPy's trust region algorithm to optimize the objective.

In [9]:
optimization = pyblp.Optimization('trust-constr', {'gtol': 1e-8, 'xtol': 1e-8})

Turning PyBLP verbosity on before optimization and off afterwards lets us monitor optimization progress in real time. We aren't adding a parameter in $\Sigma$ so we'll set that to zero (zeros mean that it will remain fixed during optimization) and we'll choose an arbitrary starting point of 1 for our new parameter in $\Pi$.

In [10]:
pyblp.options.verbose = True
mushy_results = mushy_problem.solve(sigma=0, pi=1, method='1s', optimization=optimization)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy    |   Pi:   log_income
------  ---------  |  -----  ----------
mushy   +0.00E+00  |  mushy  +1.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy    |   Pi:   log_income
------  ---------  |  -----  ----------
mushy   +0.00E+00  |  mushy    -INF    

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy    |   Pi:   log_income
------  ---------  |  -----  ----------
mushy   +0.00E+00  |  mushy    +INF    

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped  Objective   Objective   Gradient            
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares     Value    Improvement    Norm       Theta  
----  -----------  ------------  -----------  -----------  -----------  -------  ---------  -----------  ---------  ---------
 1     00:00:01         0             1           708         2187         0     +9.06E+00       

After seeing "marching down the gradient" behavior, the gradient norm is near-zero at the optimum, indicating the optimization was successful. Since the model is just identified, we also see a near-zero objective. Finally, the Hessian is positive, indicating the second order conditions are satisfied.

The new parameter is positive, indicating that wealthier individuals prefer mushy cereal. Dividing it by $\hat{\alpha}$ gives 0.25 / 30.6 = $0.01 more willingness to pay for a mushy cereal for every 1% increase in income (recall that a log regressor means we can interpret in terms of percents).

### 3. Make sure you get the same estimate with random starting values

Let's set some bounds for the new parameter and optimize from three random starting values within these bounds.

In [11]:
pi_bounds = (-10, +10)
for seed in range(3):
    initial_pi = np.random.default_rng(seed).uniform(*pi_bounds)
    seed_results = mushy_problem.solve(sigma=0, pi=initial_pi, pi_bounds=pi_bounds, method='1s', optimization=optimization)
    print(f"Initial: {initial_pi}. Estimated: {seed_results.pi[0, 0]}.")

Initial: 2.739233746429086. Estimated: 0.25128322135971737.
Initial: 0.23643249400513433. Estimated: 0.251283221361111.
Initial: -4.767757315013672. Estimated: 0.2512832207536903.


We get the same estimate each time, indicating (along with all our other diagnostics) that optimization worked well and our estimate is indeed the global minimum.

### 4. Evaluate changes to the price cut counterfactual

Let's re-run the price cut counterfactual from last exercise and compare.

In [12]:
counterfactual_data['new_shares'] = mushy_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['mushy_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data

,product_ids,mushy,prices,shares,new_prices,new_shares,iv_change,mushy_change
24,F1B04,1,0.078,6.443e-03,0.039,2.084e-02,223.638,223.522
25,F1B06,1,0.141,1.413e-01,0.141,1.392e-01,-1.450,-1.478
26,F1B07,1,0.073,8.789e-02,0.073,8.660e-02,-1.450,-1.478
27,F1B09,0,0.077,6.621e-03,0.077,6.526e-03,-1.450,-1.438
28,F1B11,0,0.167,5.427e-02,0.167,5.349e-02,-1.450,-1.438
29,F1B13,0,0.092,2.198e-02,0.092,2.166e-02,-1.450,-1.438
30,F1B17,1,0.154,1.055e-02,0.154,1.039e-02,-1.450,-1.478
31,F1B30,0,0.150,1.310e-03,0.150,1.291e-03,-1.450,-1.438
32,F1B45,0,0.147,1.052e-02,0.147,1.037e-02,-1.450,-1.438
33,F2B05,0,0.099,5.907e-02,0.099,5.822e-02,-1.450,-1.438


Results are fairly similar, which is unsurprising because the new parameter is estimated to be fairly small. Substitution from other products (and hence cannibalization) seems a bit more reasonable in the sense that we see a bit more substitution from more similar (along the mushy dimension) producucts than others. This is because high-income consumers are more likely to substitute to the now-cheaper mushy product, so products with more high-income consumers (i.e. mushy ones) are more likely to have strong reductions in demand due to substitution.

### 5. Estimate parameters on price $\times$ log income and unobserved preferences

We'll first compute the fitted values from our prive IV's first stage and verify that they are strongly correlated with prices.

In [13]:
product_data['predicted_prices'] = first_stage_results.fittedvalues
product_data[['prices', 'predicted_prices']].corr()

,prices,predicted_prices
prices,1.000,0.982
predicted_prices,0.982,1.000


Next, we'll construct an IV to target the new parameter in $\Pi$.

In [14]:
product_data['demand_instruments2'] = product_data['log_income_mean'] * product_data['predicted_prices']

We'll also construct an IV to target the new parameter in $\Sigma$.

In [15]:
def compute_differentiation(x):
    distances = x.values[:, None] - x.values[None, :]
    return np.sum(distances**2, axis=1)

product_data['demand_instruments3'] = product_data.groupby('market_ids')['predicted_prices'].transform(compute_differentiation)

Let's initialize a new problem with the new instruments.

In [16]:
product_formulations = (pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), pyblp.Formulation('0 + mushy + prices'))
agent_formulation = pyblp.Formulation('0 + log_income')
rc_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
rc_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED 
---  ----  -----  ----  ----  ---  ----  ----
94   2256  94000   1     2     1    4     2  

Formulations:
       Column Indices:             0         1   
-----------------------------  ----------  ------
 X1: Linear Characteristics      prices          
X2: Nonlinear Characteristics    mushy     prices
       d: Demographics         log_income        

Let's solve the problem, choosing a starting value for the old parameter close to its estimate and some arbitrary starting values for the two new parameters.

In [17]:
pyblp.options.verbose = True
rc_results = rc_problem.solve(
    sigma=[
        [0, 0],
        [0, 1],
    ], 
    pi=[
        [0.2],
        [1],
    ], 
    method='1s', 
    optimization=optimization,
)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy   +2.00E-01 
prices  +0.00E+00  +1.00E+00  |  prices  +1.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     -INF    
prices  +0.00E+00  +0.00E+00  |  prices    -INF    

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     +INF    
prices  +0.00E+00    +INF     |  prices    +INF    

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped  Objective   Objective     Projected                                   
Step     Time       Iterations   Evaluations  Iterations   

Again, the near-zero objective and first- and second-order condition tests indicate that we've reached a global minimum. To save time in this exercise, we won't try multiple starting values.

With an average log income of 8.09 from above, the average price coefficient is 13.7 - 5.96 * 8.09 = -34.5, which is fairly close to our old non-random price coefficient estimate $\hat{\alpha}$ of -30.6. With a standard deviation of log income of 0.9, close to the standard deviation of our $N(0, 1)$ draws, there seems to be about as much unobserved price sensitivity heterogeneity as heterogeneity from income (the two new parameter estimates are about the same magnitude). For more precise interpretations of our new parameters, we could compute elasticities for different consumer types and compare.

### 6. Evaluate changes to the price counterfactual

Let's re-run the price cut counterfactual and compare.

In [18]:
counterfactual_data['new_shares'] = rc_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['rc_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data

,product_ids,mushy,prices,shares,new_prices,new_shares,iv_change,mushy_change,rc_change
24,F1B04,1,0.078,6.443e-03,0.039,2.481e-02,223.638,223.522,285.128
25,F1B06,1,0.141,1.413e-01,0.141,1.390e-01,-1.450,-1.478,-1.621
26,F1B07,1,0.073,8.789e-02,0.073,8.627e-02,-1.450,-1.478,-1.850
27,F1B09,0,0.077,6.621e-03,0.077,6.502e-03,-1.450,-1.438,-1.808
28,F1B11,0,0.167,5.427e-02,0.167,5.346e-02,-1.450,-1.438,-1.496
29,F1B13,0,0.092,2.198e-02,0.092,2.159e-02,-1.450,-1.438,-1.759
30,F1B17,1,0.154,1.055e-02,0.154,1.038e-02,-1.450,-1.478,-1.575
31,F1B30,0,0.150,1.310e-03,0.150,1.289e-03,-1.450,-1.438,-1.556
32,F1B45,0,0.147,1.052e-02,0.147,1.036e-02,-1.450,-1.438,-1.568
33,F2B05,0,0.099,5.907e-02,0.099,5.804e-02,-1.450,-1.438,-1.735


A higher increase in demand for the product with the price cut is in-line with a slightly more negative coefficient on price. Adding a substantial amount of preference heterogeneity for price also resulted in much more reasonable substitution (and hence cannibalization) along the price dimension, in the sense that cereals with a similar price per serving as the changed one had more demand reductions than others. However, there is still very limited substitution within the mushy dimension because with no cross-market mushy variation, we couldn't credibly identify a parameter in $\Sigma$ on mushy.